In [ ]:
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True   # 확정적 연산 사용
torch.backends.cudnn.benchmark = False      # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False        # cudnn 사용 해제

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

# 데이터 준비

데이터셋 분리

In [ ]:
import pandas as pd

data_path = './data/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
# 압축 파일 코드로 풀기

from zipfile import ZipFile

with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()

with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [ ]:
from sklearn.model_selection import train_test_split

# stratify: 타겟열을 전달하며 기존 데이터에서의 타겟값 비율이 훈련 데이터와 검증 데이터 분할에도 적용됨.
train, valid = train_test_split(labels,
                                test_size=0.1,
                                stratify=labels['has_cactus'],  
                                random_state=50)

print('훈련 데이터 : ', len(train))
print('검증 데이터 : ', len(valid))

데이터셋 클래스 정의

In [ ]:
import cv2
from torch.utils.data import Dataset    # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()

        self.df = df
        self.img_dir = img_dir
        self.transform = transform  # 이미지 변환기. 이미지 전처리할 때 넘겨줘야 함
    
    def __len__(self):
        return len(self.df)
    
    # 인덱스에 해당하는 데이터 값을 반환
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[idx, 1]

        if self.transform is not None:
            image = self.transform(image)
        return image, label

데이터셋 생성

In [ ]:
from torchvision import transforms

transform = transforms.ToTensor()   # 이미지를 텐서로 변환
# 기존 shape의 모양이 뒤집힘
# 32 32 3 => 3 32 32

In [ ]:
dataset_train = ImageDataset(df=train img_dir='train/', transform=transform)
dataset_train = ImageDataset(df=valid img_dir='train/', transform=transform)